# Naver 영화 네티즌 평점 수집 - 나이트크롤러, 2020

In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

In [3]:
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=117786'
html = urlopen(url_base + url_sub)
soup = BeautifulSoup(html, 'html.parser')

In [4]:
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=117786&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [5]:
url = url_base + ifr_url
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

In [6]:
count = soup.find('strong', 'total')
count

<strong class="total">관람객 평점 <em>1,049</em>건<button class="btn_review" id="open-form-btn">내 평점 등록</button></strong>

In [7]:
count = count.find('em').string     # get_text() 대신에 string
count

'1,049'

In [8]:
count = int(count.replace(',',''))
count

1049

In [9]:
import math
total_page = math.ceil(count/10)
total_page

105

In [10]:
point = []
review = []
date = []
for page in range(1, total_page+1):
    print(page, end=' ')
    url = url_base + ifr_url + '&page=' + str(page)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    result = soup.find(class_='score_result')
    lis = result.find_all('li')
    for li in lis:
        score = li.find(class_='star_score').find('em').get_text()
        score = int(score)
        point.append(score)
        count_span = len(li.find(class_='score_reple').find(class_='').find_all('span'))
        count_icon = len(li.find_all(class_='ico_viewer'))
        count_spo = len(li.find_all(class_='text_spo'))
        criteria = str(count_span) + str(count_icon) + str(count_spo)
        #print(criteria)
        if criteria == '100':
            r = li.find(class_='score_reple').find('span').string.strip(' \t\r\n')
        elif criteria == '200':
            r = li.find(class_='score_reple').find('a')['data-src']
        elif criteria == '210':
            r = li.find(class_='score_reple').find_all('span')[1].string.strip(' \t\r\n')
        elif criteria == '310':
            r = li.find(class_='score_reple').find('a')['data-src']
        elif criteria == '311':
            r = li.find(class_='score_reple').find_all('span')[2].string.strip(' \t\r\n')
        elif criteria == '201':
            r = li.find(class_='score_reple').find_all('span')[1].string.strip(' \t\r\n')

        review.append(r)
        date.append(li.find(class_='score_reple').find_all('em')[1].get_text())

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 

In [11]:
import pandas as pd
df = pd.DataFrame({'평점':point, '일시':date, '감상평':review})
df.tail(10)

,평점,일시,감상평
1039,3,2015.02.28 00:14,뻔하고 지루하기까지 하면 무슨 말이 더 필요해?
1040,1,2018.12.16 23:37,눈빛부터 맛이 갔네 수많은 성공한 사람들이 저런 과정을 거쳐왔게지? 어이가 없다 결...
1041,1,2016.02.20 19:36,평론가 평점이 높다 이건 안봐야겠다
1042,2,2015.03.01 10:41,평점에낚임..다알바여무슨 노잼..
1043,1,2015.03.03 00:21,그냥재미없음 지루하고
1044,3,2015.02.26 20:21,진심 무슨다큐멘터리보는줄알앗음..태어나서평점처음씀
1045,1,2015.02.27 22:35,내평점 여기 리뷰쓴 사람들은 다들 영화에 조예가 깊나보네여평점보고 영화봣는데 망함ㅋ...
1046,1,2015.02.26 21:27,너무 재미없다 무슨 얘기를 하려는건지 명확하지 못하고 산만해
1047,1,2015.05.16 11:19,핵노잼입니다절대보지마세요
1048,1,2015.03.02 21:10,보다나왓다 개지루함ㅋㅋ


In [13]:
df.to_csv('Nightcrawler.csv')

In [14]:
len(df)

1049